# Tools & Libraries

In [4]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
print(file_dir)
parent_dir = file_dir.parent
print(parent_dir)
sys.path.append(str(parent_dir))

C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython\stc
C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython


In [5]:
from pa_lib.job import request_job
from pa_lib.file import project_dir, load_bin
from pa_lib.data import desc_col

In [6]:
import pandas as pd
import numpy as np

In [37]:
with project_dir("vkprog/predictions"):
    ek_list = load_bin("20200127_ek_list.feather")

17:12:14 [INFO] Started loading binary file ...
17:12:14 [INFO] Reading from file C:\Users\stc\data\vkprog\predictions\20200127_ek_list.feather
17:12:14 [INFO] ... finished loading binary file in 0.01s (0.0s CPU)


In [40]:
desc_col(ek_list)

,DTYPE,NULLS,UNIQUE
index,int64,0/12703,12703
Endkunde_NR,int64,0/12703,12703
Endkunde,object,0/12703,12130
HB_APG,object,286/12417,61
Agentur,object,7484/5219,1783
HB_Agentur,object,7527/5176,94
PLZ,int64,0/12703,2000
Ort,object,6/12697,1952
Net_2016,float64,121/12582,4506
Net_2017,float64,121/12582,4587


In [39]:
net_columns = [col for col in ek_list.columns if "Net" in col]
net_max = ek_list.loc[:, net_columns].fillna(0).max(axis="columns")
ek_list.loc[:, "Net_Max"] = net_max

In [42]:
ek_list.loc[:,net_columns+["Net_Max"]].sample(10)

,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,Net_Max
5582,0.0,11190.0,0.0,10526.0,0.0,11190.0
8935,0.0,0.0,0.0,5414.0,0.0,5414.0
9099,1500.0,1500.0,1500.0,1500.0,0.0,1500.0
5019,2736.0,0.0,0.0,0.0,0.0,2736.0
4831,2212.0,0.0,0.0,0.0,0.0,2212.0
3982,9779.0,0.0,0.0,0.0,0.0,9779.0
687,371094.0,1617353.0,1223188.0,1313408.0,304319.0,1617353.0
5897,2200.0,0.0,1941.0,1941.0,0.0,2200.0
7948,0.0,0.0,0.0,70050.0,0.0,70050.0
6838,0.0,0.0,0.0,975.0,0.0,975.0


In [68]:
ranking_pro_vb = ek_list.groupby("HB_APG")["Net_Max"].rank(ascending=True,method='dense')

ek_list.loc[:,"ranking_01"] = ranking_pro_vb


In [69]:
ek_list.loc[:,["HB_APG"]+net_columns+["Net_Max","ranking_01"]].sample(10)

,HB_APG,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,Net_Max,ranking_01
6515,PAL,76137.0,29448.0,0.0,32551.0,0.0,76137.0,670.0
10313,JLP,0.0,0.0,4590.0,0.0,0.0,4590.0,120.0
3540,DEA,14180.0,0.0,0.0,0.0,0.0,14180.0,110.0
1574,DME,9186.0,10725.0,0.0,5970.0,31148.0,31148.0,85.0
5088,RPO,0.0,0.0,0.0,849.0,672.0,849.0,71.0
11371,DCI,0.0,0.0,4041.0,0.0,0.0,4041.0,215.0
544,RPO,111979.0,58790.0,22822.0,115952.0,0.0,115952.0,352.0
4393,MOE,5612.0,0.0,1284.0,1628.0,0.0,5612.0,407.0
11698,DCI,0.0,900.0,0.0,0.0,0.0,900.0,84.0
11333,PAL,0.0,0.0,8037.0,0.0,0.0,8037.0,486.0


In [72]:
ek_list = pd.merge(
    ek_list.loc[ek_list.HB_APG == "CGA",:],
    pd.DataFrame(ek_list.groupby("HB_APG")["ranking_01"].max())
    .reset_index()
    .rename(columns={"ranking_01": "max_rank"}),
    on="HB_APG",
    how="left",
).sort_values("ranking_01",ascending=False)

In [75]:
ek_list.loc[:,"f_score"] = ek_list.ranking_01/ek_list.max_rank

In [76]:
ek_list.head(10)

,index,Endkunde_NR,Endkunde,HB_APG,Agentur,HB_Agentur,PLZ,Ort,Net_2016,Net_2017,...,VB_VK_Geb,prob_KW,Insolvenz,last_CRM_Ktkt_date,VB_FILTER_VON,VB_FILTER_BIS,Net_Max,ranking_01,max_rank,f_score
1,7,487188,Corris AG,CGA,None,None,8005,Zürich,587739.0,1309700.0,...,CGA,0.772818,None,2019-11-19,NaT,NaT,1309700.0,651.0,651.0,1.000000
0,0,108124,Rent-a-Show AG,CGA,FBM Communications AG,KUR,8032,Zürich,394976.0,503841.0,...,CGA,0.892394,None,2020-01-16,NaT,NaT,935328.0,650.0,651.0,0.998464
251,5055,100420,Hauseigentümerverband Schweiz HEV,CGA,furrerhugi.ag,BEB,8008,Zürich,39411.0,0.0,...,CGA,0.287275,None,2020-01-09,NaT,NaT,650473.0,649.0,651.0,0.996928
9,258,107524,smile.direct versicherungen,CGA,Equipe AG,MOE,8600,Dübendorf,587383.0,379658.0,...,CGA,0.549494,None,2020-01-10,NaT,NaT,587383.0,648.0,651.0,0.995392
76,1677,618071,FACE2FACE GmbH,CGA,None,None,8005,Zürich,119971.0,437115.0,...,CGA,0.396373,None,2019-10-27,NaT,NaT,437115.0,647.0,651.0,0.993856
563,10957,641359,Komitee gegen Internet-Zensur und digitale Ab,CGA,Posterscope Switzerland,NYI,8001,Zürich,0.0,0.0,...,CGA,0.089574,None,NaT,NaT,NaT,406815.0,646.0,651.0,0.992320
594,11648,499367,Turkish Airlines Inc.,CGA,Media Ltd.,MAT,8058,Zürich,0.0,376848.0,...,ROK,0.078624,None,2019-10-27,NaT,NaT,376848.0,645.0,651.0,0.990783
204,4198,135947,HELVETAS Swiss Intercooperation,CGA,Spinas Civil Voices GmbH,MOE,8021,Zürich,372230.0,237789.0,...,ROK,0.310504,None,2019-11-19,NaT,NaT,372230.0,644.0,651.0,0.989247
303,6186,116908,Zürcher Kunstgesellschaft,CGA,Kunsthaus Zürich,CGA,8024,Zürich,334900.0,276870.0,...,ROK,0.248504,None,2019-12-05,NaT,NaT,334900.0,643.0,651.0,0.987711
34,868,466249,Flughafen Zürich AG,CGA,mediaschneider ag,PKL,8058,Zürich,51250.0,121551.0,...,ROK,0.463876,None,2020-01-23,NaT,NaT,328115.0,642.0,651.0,0.986175
